# Compartmental model of a skeletal muscle

## Main reference: <a href="https://www.sciencedirect.com/science/article/pii/S0022519321000783#!"> Senneff and Lowrey, <em>Journal of Theoretical Biology </em>, 2020 </a>

### I. Sarcolemma Model

#### 1. Ionic currents
---
$$I_{ionic} = I_{Na} + I_{KDR} + I_{Cl} + I_{KIR} + I_{NaK}$$

##### 1. Sodium Current

$$I_{Na} = \bar{g}_{Na} \cdot m^3 \cdot h \cdot S \cdot (V_m - E_{Na})$$<br>
$$\frac{dS}{dt} = \frac{S_\infty - S}{\tau_S}$$ <br>
$$S_\infty = \frac{1}{1 + e^{\frac{V_m - V^\infty_S}{A^\infty_S}}}$$ <br>
$$\tau_S = \frac{60}{0.2 + 5.65 \Big(\frac{V_m + V_\tau}{100}\Big)}$$ <br>


##### 2. Potassium current

$$I_{K} = \bar{g}_K \cdot n^4 \cdot h_K \cdot (V_m - E_K)$$ <br>
$$\frac{dh_K}{dt} = \frac{h_K^\infty - h_K}{\tau_{h_K}}$$ <br>
$$h_K^\infty = \frac{1}{1 + e^{\frac{V_m - V_{h_K}^\infty}{A_{h_K}^\infty}}}$$ <br>


##### 3. Chloride Current
$$I_{Cl} = \bar{g}_{Cl} \cdot A^4 \cdot \Big(V_m - E_{Cl}\Big)$$ <br>
$$A = \frac{1}{1 + e^{\frac{V_m - V_a}{A_a}}}$$ <br>




##### Parameters for ion currents
---
Note: for initial simulations all the parameters correspond to the "Fast Twitch (FT)" muscles.

|Parameter|Unit|Definition|Value|Reference|
|:---------|:----|:----------|:-----|:-----|
|$$\bar{g}_{Na}$$| $$mScm^{-2}$$ | Maximum conductance of sodium channel|$$804$$|Senneff|
|$$\bar{g}_{K}$$ | $$mScm^{-2}$$ | Maximum conductance of potassium channel|$$64.8$$|Senneff|
|$$\bar{g}_{Cl}$$ | $$mScm^{-2}$$ | Maximum conductance of chloride channel|$$19.65$$|Senneff|
|$$G_K$$|  $$mScm^{-2}$$ |Maximum conductance of chloride channel|$$11.1$$|Senneff|
|$$\bar{J}_{NaK}$$|$$\frac{\mu \cdot mol}{cm^2 s}$$| Maximum conductance of Na-K Pump| $$6.21 \times 10^{-4}$$| Senneff|
|$$\alpha_m$$|$$\frac{1}{ms \cdot mV}$$| Maximum forward rate constant for $m$| $$0.288$$| Senneff|
|$$\beta_m$$|$$\frac{1}{ms}$$|Maximum backward rate constant for $m$|$$1.38$$|Senneff|
|$$\alpha_h$$|$$\frac{1}{ms \cdot mV}$$| Maximum forward rate constant for $h$| $$0.0081$$| Senneff|
|$$\beta_h$$|$$\frac{1}{ms}$$|Maximum backward rate constant for $m$|$$4.38$$|Senneff|
|$$\alpha_n$$|$$\frac{1}{ms \cdot mV}$$| Maximum forward rate constant for $n$| $$0.0131$$| Senneff|
|$$\beta_h$$|$$\frac{1}{ms}$$|Maximum backward rate constant for $m$|$$4.38$$|Senneff|
|$$V^m_{0.5}$$|$$\frac{1}{ms \cdot mV}$$|Half-maximal voltage for $m$|$$-46$$|Senneff|
|$$K_{\alpha_m}$$|$$\frac{1}{ms}$$|Steepness factor for $m$|$$10$$|Senneff|
|$$K_{\beta_m}$$|$$\frac{1}{ms}$$|Slope factor for $m$|$$18$$|Senneff|
|$$V_h$$|$$\frac{1}{ms}$$|Half maximal voltage for $h$|$$-45$$|Senneff|
|$$K_{\alpha_h}$$|$$\frac{1}{ms}$$|Steepness factor for $h$|$$14.7$$|Senneff|
|$$K_{\beta_h}$$|$$\frac{1}{ms}$$|Slope factor for $h$|$$9$$|Senneff|
|$$V^\infty_S$$|$$mV$$|Half maximal voltage for $S$|$$-78$$|Senneff|
|$$V_\tau$$|$$mV$$|Half maximal voltage for $S$|$$90$$|Senneff|
|$$A^\infty_S$$|$$mV$$|Steepness factor for $S$|$$5.8$$|Senneff|
|$$V_n$$|$$mV$$|Half maximal voltage for $n$|$$-40$$|Senneff|
|$$K_{\alpha_n}$$|$$mV$$|Steepness factor for $n$|$$7$$|Senneff|
|$$K_{\beta_n}$$|$$mV$$|Slope factor for $n$|$$40$$|Senneff|
|$$V_a$$|$$mV$$|Half-maximal voltage for $a$|$$70$$|Senneff|
|$$A_a$$|$$mV$$|Steepness factor for $a$|$$150$$|Senneff|
|$$E_{Na}$$|$$mV$$|Sodium channel Nernst Potential|$$59.3$$|Wallinga|
|$$E_{K}$$|$$mV$$|Potassium channel Nernst potential|$$-81.8$$|Wallinga|
|$$E_{Cl}$$|$$mV$$|Chloride channel Nernst potential|$$-78.3$$|Wallinga|

In [22]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
from scipy.integrate import solve_ivp, odeint
import os
sns.set_style("ticks")
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["figure.dpi"] = 100
qcolors = ["#C6D57E", "#D57E7E", "#A2CDCD", "#FFE1AF"]
qPal = sns.color_palette(sns.set_palette(qcolors))

In [26]:
class Skeletal_muscle:
    """ This class contains the parameters and equations describing dynamics of a skeletal muscle. The class is broken into following parts:
    1. Sarcolemma Model: - Contains ionic currents modelled with Hodgkin-Huxley-like conductance based equations. (UNDER CONSTRUCTION)
    2. Transverse tubule (T-tubule) Model: - Containts the T-tubule ion channels and voltage-gated calcium channels. (UNDER CONSTRUCTION)
    3. Sarcoplasmic Reticulum: - Consists of ryanodine receptors, IP3 receptors and SERCA Pumps. (UNDER CONSTRUCTION)
    4. Mitochondria (UNDER CONSTRUCTION)
    """
    